<a href="https://colab.research.google.com/github/Saifullah785/langchain-generative-ai-journey/blob/main/Lecture_15_tool_calling_in_langchain/Lecture_15_tool_calling_in_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries for LangChain, HuggingFace integration, and requests.
# !pip install -q langchain_huggingface langchain-core requests

In [2]:
# Import necessary classes and functions from LangChain and requests library.
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [4]:
# Import userdata from google.colab to access secrets.
from google.colab import userdata

# Get the Hugging Face API key from Colab secrets, named "hf_token".
hf_api_key = userdata.get("hf_token")

In [18]:
# Initialize the HuggingFaceEndpoint with the Llama-3.1-8B-Instruct model for text generation.
llm = HuggingFaceEndpoint(
    # repo_id="meta-llama/Meta-Llama-3-8B-Instruct", # Example of a commented-out model
    repo_id='meta-llama/Llama-3.1-8B-Instruct',
)

# Wrap the endpoint in a ChatHuggingFace object for chat-style interaction.
model = ChatHuggingFace(llm=llm)

In [20]:
# Define a tool function using the @tool decorator.
# This function takes two integers, a and b, and returns their product.
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [21]:
# Invoke the multiply tool with inputs a=2 and b=3 and print the result.
print(multiply.invoke({"a": 2, "b": 3}))

6


In [22]:
# Access and display the name of the multiply tool.
multiply.name

'multiply'

In [23]:
# Access and display the description of the multiply tool.
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [24]:
# Access and display the arguments expected by the multiply tool.
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [25]:
# Invoke the chat model with the input 'hi' and display the response.
model.invoke('hi')

AIMessage(content="It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 36, 'total_tokens': 59}, 'model_name': 'meta-llama/Llama-3.1-8B-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run--b65e8c94-f071-4cf0-a43f-3dfab0b22d08-0', usage_metadata={'input_tokens': 36, 'output_tokens': 23, 'total_tokens': 59})

In [26]:
# Bind the multiply tool to the chat model, creating a new model instance that can use the tool.
llm_with_tools = model.bind_tools([multiply])

In [28]:
# Example of how to invoke the model with tools (currently commented out).
# llm_with_tools.invoke('hi how are you')